In [21]:
import pandas as pd 
import os 
import numpy as np 

In [22]:
centro = '8811456' #input("Introduza o centro: ")

In [23]:

folder_path = 'C:\\Users\\1420844\\Downloads\\dados_centros'

files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]


combined_df = pd.DataFrame()


for file in files:
    file_path = os.path.join(folder_path, file)
    
    
    df = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    
    
    combined_df = pd.concat([combined_df, df], ignore_index=True)



In [24]:
# -- limpeza do dataset 
df = combined_df
df.columns=[x.lower().strip() for x in df.columns]
df.drop_duplicates()
df['centro'] = df['centro'].astype(str)
df['centro'] = df['centro'].str.strip()
df['giro'] = df['giro'].str.replace(' ', '').str.strip().str[:5]
df['cp'] = df['cp'].str.replace(' ', '').str.strip()
df['longitude'] = df['longitude'].str.replace('\n', '').str.strip()
df=df[df.latitude != '0']
df=df[df.latitude != '0.0']
df = df[df['latitude'] != '']
df=df[df.cod_t_even == 'I']
df = df[df['cp'].str.len() >= 8]

In [25]:
df=df[df['centro']==centro]

In [26]:
# validar os giros que devem ser usados no pit stop

giro_unique_ocorrencias = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_dias')

# Contar o número de ocorrências únicas por 'loptica'
loptica_unique_ocorrencias = df.groupby('giro')['loptica'].nunique().reset_index(name='N_objetos')

giros_pit_stop = pd.merge(giro_unique_ocorrencias, loptica_unique_ocorrencias, left_on='giro', right_on='giro', how='inner')

In [27]:

# downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")


# file_path = os.path.join(downloads_path, f'{centro}.xlsx')
# giros_pit_stop.to_excel(file_path, index=False)

# print(f"File saved at: {file_path}")

In [28]:
giros_pit_stop

,giro,N_dias,N_objetos
0,2405A,6,512
1,2405B,6,391
2,2405C,5,448
3,2405D,1,3
4,2410A,6,666
...,...,...,...
81,DIS05,1,1
82,LOCAL,5,166
83,LR001,1,1
84,REDE+,1,1
